# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
loans_binary_encoded = pd.get_dummies(df, columns=["home_ownership", "verification_status", "pymnt_plan", "initial_list_status", "application_type", "hardship_flag", "debt_settlement_flag"])
loans_binary_encoded['issue_month'] = pd.DatetimeIndex(loans_binary_encoded['issue_d']).month
loans_binary_encoded['issue_year'] = pd.DatetimeIndex(loans_binary_encoded['issue_d']).year
loans_binary_encoded['next_pymnt_month'] = pd.DatetimeIndex(loans_binary_encoded['next_pymnt_d']).month
loans_binary_encoded['next_pymnt_year'] = pd.DatetimeIndex(loans_binary_encoded['next_pymnt_d']).year
loans_binary = loans_binary_encoded.drop(columns=['issue_d', 'next_pymnt_d'])

## Split the Data into Training and Testing

In [6]:
# Create our features
X = loans_binary.drop(columns='loan_status')

# Create our target
y = loans_binary.loan_status

In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,issue_month,issue_year,next_pymnt_month,next_pymnt_year
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.0,68817.000000,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.123879,0.876121,0.860340,0.139660,1.0,1.0,1.726172,2019.0,4.616839,2019.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.329446,0.329446,0.346637,0.346637,0.0,0.0,0.743862,0.0,0.486161,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.000000,2019.0,4.000000,2019.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,1.000000,0.000000,1.0,1.0,1.000000,2019.0,4.000000,2019.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.000000,1.000000,0.000000,1.0,1.0,2.000000,2019.0,5.000000,2019.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.000000,1.000000,0.000000,1.0,1.0,2.000000,2019.0,5.000000,2019.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.0,1.0,3.000000,2019.0,5.000000,2019.0


In [8]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [12]:
# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=500, random_state=1)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)


In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.9121185701830863


In [15]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,60,27
Actual 1,1485,15633


In [16]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.69      0.91      0.07      0.79      0.62        87
   low_risk       1.00      0.91      0.69      0.95      0.79      0.64     17118

avg / total       0.99      0.91      0.69      0.95      0.79      0.64     17205



In [17]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08021862004459797, 'total_rec_prncp'),
 (0.0608190084251239, 'total_rec_int'),
 (0.056979622075916815, 'total_pymnt'),
 (0.0566829932706354, 'total_pymnt_inv'),
 (0.05644923168509379, 'last_pymnt_amnt'),
 (0.028300440007455466, 'issue_month'),
 (0.026314753309087568, 'int_rate'),
 (0.018986950913292597, 'dti'),
 (0.017036330142033713, 'mths_since_recent_inq'),
 (0.01683392760886929, 'installment'),
 (0.016625410148668824, 'annual_inc'),
 (0.016264876850381627, 'bc_util'),
 (0.015982763605404335, 'max_bal_bc'),
 (0.015908764980613128, 'il_util'),
 (0.015265829518845577, 'mo_sin_old_il_acct'),
 (0.015020323205040726, 'mo_sin_old_rev_tl_op'),
 (0.014761208007274621, 'out_prncp'),
 (0.01466271954218957, 'avg_cur_bal'),
 (0.01417540757038909, 'revol_bal'),
 (0.014145362423737795, 'out_prncp_inv'),
 (0.014079865465522826, 'total_bc_limit'),
 (0.01403538067683539, 'all_util'),
 (0.013942508366374224, 'tot_cur_bal'),
 (0.013726028030158079, 'bc_open_to_buy'),
 (0.013667629582948187, 'tot_h

### Easy Ensemble Classifier

In [18]:
# Train the Classifier
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test.ravel())))
    print()
    
# Choose a learning rate and create classifier
classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.05,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=1)

# Fit the model
classifier.fit(X_train_scaled, y_train.ravel())

# Make Prediction
predictions = classifier.predict(X_test_scaled)

Learning rate:  0.05
Accuracy score (training): 0.996
Accuracy score (validation): 0.996

Learning rate:  0.1
Accuracy score (training): 0.996
Accuracy score (validation): 0.995

Learning rate:  0.25
Accuracy score (training): 0.994
Accuracy score (validation): 0.993

Learning rate:  0.5
Accuracy score (training): 0.995
Accuracy score (validation): 0.995

Learning rate:  0.75
Accuracy score (training): 0.995
Accuracy score (validation): 0.994

Learning rate:  1
Accuracy score (training): 0.995
Accuracy score (validation): 0.995



In [19]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.995931415286254


In [20]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,32,55
Actual 1,15,17103


In [21]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.68      0.37      1.00      0.48      0.61      0.34        87
   low_risk       1.00      1.00      0.37      1.00      0.61      0.39     17118

avg / total       1.00      1.00      0.37      1.00      0.61      0.39     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Gradient Boosting Classifier has the better accuracy score.

2. Which model had the best recall score?

    The Random Forest model has better recall score for high risk class while the Gradient Boosting Classifier does better for low risk class.

3. Which model had the best geometric mean score?

    The Random Forest model has the better geo avg.

4. What are the top three features?

    The top 3 features are Total Received Principal, Total Received Interest, and Total Payment as per the importance report. 